In [ ]:
from waffles.np04_analysis.lightyield_vs_energy.scripts.MyAnaPeak import MyAnaPeak
from waffles.np04_analysis.lightyield_vs_energy.scripts.utils_plots import *
from scipy.signal import find_peaks
import importlib

In [ ]:
FS_filepath = "/afs/cern.ch/work/a/anbalbon/public/reading_rucio/run027361/run027361_full_streaming_no_selection_1st.pkl"
print("Loading Full-Streaming Wfset...")
with open(FS_filepath, 'rb') as f:
    FS_wfset = pickle.load(f) 
    print('FS done\n')

In [ ]:
ST_filepath = "/afs/cern.ch/work/a/anbalbon/public/reading_rucio/run027361/run027361_self_trigger_no_selection_1st.pkl"
print("Loading Self-Trigger Wfset...")
with open(ST_filepath, 'rb') as f:
    ST_wfset = pickle.load(f) 
    print('ST done\n')   

In [ ]:
import waffles.np04_analysis.lightyield_vs_energy.scripts.MyAnaPeak as MyAnaPeak_module
importlib.reload(MyAnaPeak_module)
from waffles.np04_analysis.lightyield_vs_energy.scripts.MyAnaPeak import MyAnaPeak

analysis_label = 'test_peak_finding'
analysis_class= MyAnaPeak

ip = IPDict(baseline_limits= [0, 50],
            int_ll = 55,
            int_ul = 100,
            amp_ll = 55,
            amp_ul= 100,
            invert = False,
            baseline_method = "EasyMedian"
            )

_ = ST_wfset.analyse(label=analysis_label ,analysis_class=analysis_class, input_parameters=ip, checks_kwargs = IPDict({'points_no': ST_wfset.points_per_wf}), overwrite=True)
_ = FS_wfset.analyse(label=analysis_label ,analysis_class=analysis_class, input_parameters=ip, checks_kwargs = IPDict({'points_no': FS_wfset.points_per_wf}), overwrite=True)

print('done\n\n')

In [ ]:
plotting_overlap_wf_PEAK_NEW(FS_wfset, n_wf= 20, show  = True, save = False, peak_bool = True, peak_beam_bool = True, analysis_label=analysis_label)

In [ ]:
FS_offset_list = []
FS_beam_peak_absolute_time = []
for wf in FS_wfset.waveforms:
    FS_offset_list.extend(wf.analyses[analysis_label].result['time_offset'])
    FS_beam_peak_absolute_time.extend(wf.analyses[analysis_label].result['beam_peak_absolute_time'])

fig = go.Figure(
    data=[
        go.Histogram(
            x=FS_offset_list,
            nbinsx=20000,
            marker_color='royalblue',
            opacity=0.7
        )
    ]
)

fig.show()


In [ ]:
ST_offset_list = []
ST_beam_peak_absolute_time = []
for wf in ST_wfset.waveforms:
    ST_offset_list.extend(wf.analyses[analysis_label].result['time_offset'])
    ST_beam_peak_absolute_time.extend(wf.analyses[analysis_label].result['beam_peak_absolute_time'])

fig = go.Figure(
    data=[
        go.Histogram(
            x=ST_offset_list,
            nbinsx=20000,
            marker_color='royalblue',
            opacity=0.7
        )
    ]
)

fig.show()

In [ ]:
# Filtro per selezionare solo i beam events (sia FS che ST)
from waffles.data_classes.Waveform import Waveform
def beam_filter(waveform: Waveform, analysis_label) -> bool:
    if len(waveform.analyses[analysis_label].result['beam_peak_index'])>0:
        return True
    else:
        return False

In [ ]:
from waffles.data_classes.WaveformSet import WaveformSet
FS_beam_wfset = WaveformSet.from_filtered_WaveformSet(FS_wfset, beam_filter, analysis_label)
print('FS events: ', len(FS_wfset.waveforms))
print('FS beam events:', len(FS_beam_wfset.waveforms))

print()

ST_beam_wfset = WaveformSet.from_filtered_WaveformSet(ST_wfset, beam_filter, analysis_label)
print('FST events: ', len(ST_wfset.waveforms))
print('ST beam events:', len(ST_beam_wfset.waveforms))

In [ ]:
plotting_overlap_wf_PEAK_NEW(FS_beam_wfset, n_wf= 20, show  = True, save = False, peak_bool = False, peak_beam_bool = True, analysis_label=analysis_label)

In [ ]:
plotting_overlap_wf_PEAK_NEW(ST_beam_wfset, n_wf= 20, show  = True, save = False, peak_bool = False, peak_beam_bool = True, analysis_label=analysis_label)

In [ ]:

attr_list = [
    "timestamp",
    "daq_window_timestamp",
    "endpoint",
    "channel",
    "time_offset",
]

for i in range(10):
    FS_wf = FS_beam_wfset.waveforms[i]
    ST_wf = ST_beam_wfset.waveforms[i]

    print(f"\n======= Waveform {i} =======")

    for label, wf in [("FS", FS_wf), ("ST", ST_wf)]:
        print(f"\n{label} waveform:")
        for attr in attr_list:
            print(f"  {attr:20s}: {getattr(wf, attr)}")

In [ ]:
def endpoint_filter(waveform : Waveform, end : int,) -> bool:
    if (waveform.endpoint == end) :
        return True
    else:
        return False
    
def channel_filter(waveform : Waveform, end : int, ch : int) -> bool:
    if (waveform.channel == ch) and (waveform.endpoint == end) :
        return True
    else:
        return False


In [ ]:
def remove_close_values(values, delta = 200):
    if not values:
        return []

    values = sorted(values)
    filtered = [values[0]]

    for v in values[1:]:
        if abs(v - filtered[-1]) > delta:
            filtered.append(v)

    return filtered


In [ ]:
all_peak_absolute_time = sorted(ST_beam_peak_absolute_time + FS_beam_peak_absolute_time)
print('Number of beam peaks before removing close values (total): ', len(all_peak_absolute_time))
print('Number of beam peaks before removing close values (FS): ', len(FS_beam_peak_absolute_time))
print('Number of beam peaks before removing close values (ST): ', len(FS_beam_peak_absolute_time))
print()


delta_ts = 100 #100 sembra essere abbastanza
cleaned_peak_absolute_time = remove_close_values(all_peak_absolute_time, delta=delta_ts)
print("Number of beam peaks after removing close values (total): ", len(cleaned_peak_absolute_time))

In [ ]:
def timestamp_filter(waveform : Waveform, analysis_label : str, ts : int, ts_delta : int = 200) -> bool:
    bool_return = False
    for single_peak_absolute_time in waveform.analyses[analysis_label].result['beam_peak_absolute_time']:
        if abs(single_peak_absolute_time - ts) < ts_delta :
            bool_return = True
    return bool_return

In [ ]:
from integral_computation_function import channel_integral_computation
from waffles.np04_data.ProtoDUNE_HD_APA_maps import APA_map # apa map
from waffles.data_classes.ChannelWsGrid import ChannelWsGrid # channel grid


In [ ]:
i=0
for beam_trigger_time in cleaned_peak_absolute_time:
    try:
        trigger_FS_beam_wfset = WaveformSet.from_filtered_WaveformSet(FS_beam_wfset, timestamp_filter, analysis_label, beam_trigger_time, delta_ts)
        # Integration...
        apa = 1
        integration_analysis_label = 'integrator'
        period = 'june'
        grid_apa_FS = ChannelWsGrid(APA_map[apa], trigger_FS_beam_wfset, compute_calib_histo=False)
        for endpoint in grid_apa_FS.ch_wf_sets.keys():
            for channel in grid_apa_FS.ch_wf_sets[endpoint].keys():
                grid_apa_FS.ch_wf_sets[endpoint][channel] = channel_integral_computation(grid_apa_FS.ch_wf_sets[endpoint][channel], period = period, integration_analysis_label = integration_analysis_label)

        trigger_ST_beam_wfset = WaveformSet.from_filtered_WaveformSet(ST_beam_wfset, timestamp_filter, analysis_label, beam_trigger_time, delta_ts)
        # Integration...
        apa = 2
        integration_analysis_label = 'integrator'
        period = 'june'
        grid_apa_ST = ChannelWsGrid(APA_map[apa], trigger_ST_beam_wfset, compute_calib_histo=False)
        for endpoint in grid_apa_ST.ch_wf_sets.keys():
            for channel in grid_apa_ST.ch_wf_sets[endpoint].keys():
                grid_apa_ST.ch_wf_sets[endpoint][channel] = channel_integral_computation(grid_apa_ST.ch_wf_sets[endpoint][channel], period = period, integration_analysis_label = integration_analysis_label)
                
        
        print(f"{i} , {beam_trigger_time}--> FS: {len(trigger_FS_beam_wfset.waveforms)}\t ST: {len(trigger_ST_beam_wfset.waveforms)}")
        
    except:
        print('skip')
        continue

    i+=1

# grid_apa = grid_apa_FS
# print(f"APA {apa} - endpoint {endpoint} - channel {channel}\nSPE mean amplitude: {grid_apa.ch_wf_sets[endpoint][channel].waveforms[0].analyses[integration_analysis_label].result['gain']:.2f} \nIntegration limits: {grid_apa.ch_wf_sets[endpoint][channel].waveforms[0].analyses[integration_analysis_label].result['integration_limits']}")
# print(f"Integral beam wf example: {grid_apa.ch_wf_sets[endpoint][channel].waveforms[0].analyses[integration_analysis_label].result['integral']:.2f}")
# print(f"# Photoelectrons: {grid_apa.ch_wf_sets[endpoint][channel].waveforms[0].analyses[integration_analysis_label].result['integral_pe']}")

# plotting_overlap_wf_PEAK_NEW(grid_apa.ch_wf_sets[endpoint][channel], n_wf = 10, show = True, save = False, x_min=None, x_max=None, y_min=None, y_max=None, int_ll=grid_apa.ch_wf_sets[endpoint][channel].waveforms[0].analyses[integration_analysis_label].result['integration_limits'][0], int_ul=grid_apa.ch_wf_sets[endpoint][channel].waveforms[0].analyses[integration_analysis_label].result['integration_limits'][1], analysis_label = integration_analysis_label)
            

In [ ]:
prova_FS = WaveformSet.from_filtered_WaveformSet(ST_beam_wfset, timestamp_filter, analysis_label, beam_trigger_time, 107438565474668537)
plotting_overlap_wf_PEAK_NEW(prova_FS, n_wf= 2, show  = True, save = False, peak_bool = False, peak_beam_bool = True, analysis_label=analysis_label)


In [ ]:
# SEMBRA FUNZIONARE!!!! 
# 
# Perhcè vedo 30 waveform in APA 1 e ocsì poche in APA2 ? DA RISOLVERE!
# 

# DA FARE:
# da implementare il calcolo dell'integrale per APA 1 

In [ ]:
# da usare se non mostra i grafici

import plotly.io as pio
pio.renderers.default = "notebook"
